In [1]:
import cv2
import numpy as np
import glob
import time

In [2]:
f = open("forms.txt",'r')
FileNames = []
WriterID = []
for i in range(16):
    next(f,None)
for line in f:
    Splitted = line.split(sep=' ')
    FileNames.append(Splitted[0])
    WriterID.append(Splitted[1])


f.close()

In [3]:
len(FileNames)

1539

In [4]:
Dataset_FileName = 'F:\\Pattern_Proj_Dataset\\Form\\'
Dataset_Files = 'F:\\Pattern_Proj_Dataset\\Form\\*.png'

def ShowImage(name,formName='image'):
    res = cv2.resize(name,(768,768))
    cv2.imshow(formName,res)
    cv2.waitKey(0);
    
def returnFilePath(name):
    return Dataset_FileName + name

def CompareToImage(img,img2,name1='image1',name2='image2'):
    res = cv2.resize(img,(768,768))
    cv2.imshow(name1,res)
    ShowImage(img2,name2)


In [12]:
img = cv2.imread(returnFilePath('a01-000u.png'),0)
ShowImage(img)

In [7]:
blur_gauss = cv2.GaussianBlur(img,(3,3),0)
_ , threshed = cv2.threshold(blur_gauss , 127 , 255 , cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
_ , threshed_nogauss = cv2.threshold(img , 127 , 255 , cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
CompareToImage(threshed,threshed_nogauss,"Gauss","NoGauss")

In [15]:
# houghlines variables

rho = 10 # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 1  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 350  # minimum number of pixels making up a line
max_line_gap = 5# maximum gap in pixels between connectable line segments

In [18]:
def GetLinesP(img):
    blur_gauss = cv2.GaussianBlur(img,(5,5),0)
    _ , threshed = cv2.threshold(blur_gauss , 200 , 255 , cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
    ShowImage(threshed)
    line_image = np.copy(img) * 0  # creating a blank to draw lines on
    
    lines = cv2.HoughLinesP(threshed, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)
    return lines , line_image
def drawLines(lines , imgToDrawOn):
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(imgToDrawOn,(x1,y1),(x2,y2),(255,0,0),5)
def printLines(lines):
    print(lines.shape[0])
    for line in lines:
        for x1,y1,x2,y2 in line:
            print("Start : ( {} , {} ) , End : ( {} , {} ) ".format(x1,y1,x2,y2))

In [10]:
def AllFunctionalitiesP(img,printLine = False ):

    lines , line_image = GetLinesP(img)
    drawLines(lines,line_image)
    if(printLine) : printLines(lines)
    lines_edges = cv2.addWeighted(img, 0.8, line_image, 1, 0)

    return lines_edges , lines


In [17]:
imgPrint , _ = AllFunctionalitiesP(img,True)
ShowImage(imgPrint)

NameError: name 'line_image' is not defined

In [10]:
imgPrint ,_ = AllFunctionalitiesP(img[400:,300:],printLine=True)
ShowImage(imgPrint)

3
Start : ( 844 , 2390 ) , End : ( 1597 , 2390 ) 
Start : ( 861 , 205 ) , End : ( 1664 , 205 ) 
Start : ( 1067 , 204 ) , End : ( 1864 , 204 ) 


In [11]:
img2 = cv2.imread(returnFilePath('b06-068.png'),0)
ShowImage(img2)

In [13]:
imgPrint , _ = AllFunctionalitiesP(img2[400:,300:],printLine=True)
ShowImage(imgPrint)

2
Start : ( 47 , 2398 ) , End : ( 1851 , 2398 ) 
Start : ( 45 , 257 ) , End : ( 1847 , 257 ) 


In [20]:
img3 = cv2.imread(returnFilePath('d04-016.png'),0)
ShowImage(img3)

In [21]:
imgPrint = AllFunctionalities(img3[400:,300:],printLine=True)
ShowImage(imgPrint)

2
Start : ( 52 , 213 ) , End : ( 1849 , 213 ) 
Start : ( 946 , 2404 ) , End : ( 1850 , 2404 ) 


In [52]:
LineCheck = []
start = time.time()
for file in glob.glob(Dataset_Files):
    img = cv2.imread(file,0)

    imgPrint = AllFunctionalitiesP(img[400:,300:]) 

print("Functionality in {} ".format((time.time() - start)))
    

Functionality in 261.9237630367279 


In [53]:
LineChecknp = np.array(LineCheck) 

In [54]:
fname_np = np.array(FileNames)

In [55]:
np.unique(LineChecknp , return_counts=True) #kernel 5x5

(array([1, 2, 3, 4, 5, 6]), array([  2, 599, 549, 368,  19,   2], dtype=int64))

In [56]:
only2lines = LineChecknp == 1
lines2Img = fname_np[only2lines]

In [58]:
for fname in lines2Img:
    fname = fname+ '.png'
    img = cv2.imread(returnFilePath(fname),0)
    imgPrint = AllFunctionalitiesP(img[400:,300:])
    ShowImage(imgPrint)


# Trying the blob detector

In [12]:
ShowImage(img)

In [13]:
imgPrint  , _= AllFunctionalitiesP(img[400:,300:],printLine=True)
ShowImage(imgPrint)

3
Start : ( 844 , 2390 ) , End : ( 1597 , 2390 ) 
Start : ( 861 , 205 ) , End : ( 1664 , 205 ) 
Start : ( 1067 , 204 ) , End : ( 1864 , 204 ) 


In [34]:
Y_Crop = 400 + 205 + 15 + 50
Y_Crop2 = 400 + 2390 - 700
CroppedImage = img[Y_Crop:Y_Crop2,300:]
ShowImage(CroppedImage)

In [64]:
params = cv2.SimpleBlobDetector_Params()





In [69]:
blob = cv2.SimpleBlobDetector_create()

In [70]:
keypoints = blob.detect(CroppedImage)
len(keypoints)

13

In [71]:
CroppedImageColored = cv2.cvtColor(CroppedImage,cv2.COLOR_GRAY2BGR)

In [72]:
im_with_keypoints = cv2.drawKeypoints(CroppedImageColored,keypoints,np.array([]) , (0,0,255) , cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)

In [73]:
ShowImage(im_with_keypoints)

In [107]:
# ## (2) threshold
# th, threshed2 = cv2.threshold(CroppedImage, 127, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)


# pts = cv2.findNonZero(threshed2)
# rect = cv2.minAreaRect(pts)

# (cx,cy),(w,h),ang = rect

# if (w>h):
#     w,h = h , w
#     ang += 90
# # (4) Find rotated matrix, do rotation
# M = cv2.getRotationMatrix2D((cx,cy), ang, 1.0)
# rotated = cv2.warpAffine(CroppedImage, M, (img.shape[1], img.shape[0]))

# ## (5) find and draw the upper and lower boundary of each lines
# hist = cv2.reduce(rotated,1, cv2.REDUCE_AVG).reshape(-1)

# th = 2
# H,W = img.shape[:2]
# uppers = [y for y in range(H-1) if hist[y]<=th and hist[y+1]>th]
# lowers = [y for y in range(H-1) if hist[y]>th and hist[y+1]<=th]

# for y in uppers:
#     cv2.line(rotated, (0,y), (W, y), (255,0,0), 5)

# for y in lowers:
#     cv2.line(rotated, (0,y), (W, y), (0,255,0), 5)

# cv2.imwrite("result.png", rotated)
# ShowImage(rotated)



In [92]:
cy = int(cy)
cx = int(cx)
w = int(w)
h = int(h)

In [101]:
ShowImage(rotated[cy-round(h/2):cy+round(h/2),cx-round(w/2):cx+round(w/2)])

In [96]:
ShowImage(rotated)